## Criterios para partición en árboles de clasificación

Cuando trabajamos con árboles de decisión en clasificación, debemos decidir cómo dividir los datos para que los grupos resultantes sean lo más "puros" posible. A diferencia de regresión, donde se minimiza la **varianza conjunta**, en clasificación usamos medidas como:

- Índice de GINI
- Entropía
- Log Loss (Pérdida logarítm)

---

### 1. GINI (Índice de Impureza de Gini)

**¿Qué es?**  
El índice GINI mide qué tan mezcladas están las clases dentro de un grupo. Cuanto más mezcladas, mayor es la impureza.

**Ejemplo simple:**  
Si en un grupo hay 50% rojas y 50% azules, GINI es alto.  
Si hay solo una clase, GINI es 0.$$
\text{GINI} = 1 - \sum_{i=1}^{n} p_i^2
$$um_{i=1}^{n} p_i^2
\]

Donde \( p_i \) es la proporción de la clase \( i \).

**¿Cómo se usa?**  
Al dividir los datos en un árbol, se busca la división que reduzca el GINI total, es decir, que dejos grupos más "puros".

---

### 2. Entropía

**¿Qué es?**  
La entropía mide la incertidumbre o desorden en un grupo. A más incertidumbre, mayor entropía.

**Ejemplo simple:**  
Si tienes 50% de cada clase y no sabes cuál aparecerá, la entropía es alta.  
Si el 100% de los casos son de una sola clase, la $$
\text{Entropía} = - \sum_{i=1}^{n} p_i \cdot \log_2(p_i)
$$
 - \sum_{i=1}^{n} p_i \cdot \log_2(p_i)
\]

**¿Cómo se usa?**  
Usada en algoritmos como ID3 o C4.5. Se elige la partición que más reduzca la entropía después de dividir (mayor "ganancia de información").

---

### 3. Log Loss (Pérdida logarítmica)

**¿Qué es?**  
Log Loss mide el error de las predicciones probabilísticas. Penaliza más cuando se hace una predicción con mucha seguridad y se falla.

**Ejemplo simple:**  
Si tu modelo dice "99% seguro que es clase A" y es clase B, la pen$$
\text{Log Loss} = - [y \cdot \log(p) + (1 - y) \cdot \log(1 - p)]
$$g Loss} = - [y \cdot \log(p) + (1 - y) \cdot \log(1 - p)]
\]

Donde:
- \( y \) es el valor real (0 o 1)
- \( p \) es la probabilidad predicha para la clase 1

**¿Cómo se usa?**  
No se usa directamente para dividir en los árboles, sino para evaluar modelos que dan probabilidades (como la regresión logística o los árboles con estimación probabilística en hojas).

---

### Diferencia entre Entropía y Log Loss

| Característica         | Entropía                             | Log Loss                               |
|------------------------|--------------------------------------|----------------------------------------|
| ¿Dónde se usa?         | Para dividir datos en árboles        | Para evaluar modelos probabilísticos   |
| ¿Qué mide?             | Incertidumbre del conjunto           | Precisión de una predicción individual |
| ¿Usa probabilidades?   | Sí, de las clases del conjunto       | Sí, de la predicción del modelo        |
| ¿Optimiza división?    | Sí                                   | No                                     |

**Conclusión:**  
- GINI y Entropía se usan para **decidir por dónde dividir** los datos en clasificación.
- Log Loss se usa para **evaluar modelos probabilísticos**, no para dividir directamente.


In [61]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

In [63]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [65]:
# Variable de partición usando hp 
# hp <= 100 en un grupo, hp > 100 en el otro
grupo_izq = df[df['hp'] <= 100]
grupo_der = df[df['hp'] > 100]

print("Tamaño grupo izquierda (hp <= 100):", len(grupo_izq))
print("Tamaño grupo derecha (hp > 100):", len(grupo_der))


Tamaño grupo izquierda (hp <= 100): 9
Tamaño grupo derecha (hp > 100): 23


In [67]:
# Funciones para calcular GINI y Entropía

def gini(grupo):
    clases = grupo['am'].value_counts(normalize=True)
    return 1 - np.sum(clases**2)

def entropia(grupo):
    clases = grupo['am'].value_counts(normalize=True)
    return -np.sum([p * np.log2(p) for p in clases if p > 0])


In [69]:
# Calculo de GINI y Entropía para cada grupo
gini_izq = gini(grupo_izq)
gini_der = gini(grupo_der)

ent_izq = entropia(grupo_izq)
ent_der = entropia(grupo_der)

# Peso de cada grupo
n_total = len(df)
peso_izq = len(grupo_izq) / n_total
peso_der = len(grupo_der) / n_total

# GINI total ponderado
gini_total = peso_izq * gini_izq + peso_der * gini_der

# Entropía total ponderada
entropia_total = peso_izq * ent_izq + peso_der * ent_der

print("GINI total:", round(gini_total, 4))
print("Entropía total:", round(entropia_total, 4))


GINI total: 0.4293
Entropía total: 0.8955


In [71]:
# Calcular Log Loss (pérdida logarítmica) para cada grupo

def logloss_aprox(grupo):
    p = grupo['am'].mean()  # probabilidad promedio de clase 1 (manual)
    y_true = grupo['am']
    y_pred = np.full(len(grupo), p)  # se predice la misma probabilidad para todo el grupo
    return log_loss(y_true, y_pred, labels=[0, 1])

logloss_izq = logloss_aprox(grupo_izq)
logloss_der = logloss_aprox(grupo_der)

logloss_total = peso_izq * logloss_izq + peso_der * logloss_der

print("Log Loss total:", round(logloss_total, 4))


Log Loss total: 0.6207
